# InvarLock: Compare & Certify (BYOE)

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/invarlock/invarlock/blob/main/notebooks/invarlock_compare_certify.ipynb)

**Purpose:** Run the recommended baseline-vs-subject certification workflow (Compare & Certify / BYOE).
**Runtime:** ~10 minutes (CPU; faster on GPU)
**Requires:** `invarlock[hf]` and network for first downloads

This notebook demonstrates:

- Deterministic pairing (baseline windows pinned for the subject run)
- Certificate generation (`evaluation.cert.json`)
- Verification (`invarlock verify`) and HTML export

> For real BYOE: set `--baseline` to your original checkpoint and `--subject` to your externally-edited checkpoint.


In [ ]:
!pip -q install "invarlock[hf]"
import sys
import invarlock

print("Python:", sys.version.split()[0])
print("InvarLock:", invarlock.__version__)


In [ ]:
!invarlock doctor --json || true


## Demo config (small window counts)

To keep this notebook fast, we create:

- A small preset (`demo_preset.yaml`) with modest preview/final window counts
- A demo edit config (`quant_rtn_demo.yaml`) using the built-in `quant_rtn` edit

For real evidence, remove `INVARLOCK_TINY_RELAX=1` and use `--profile ci` or `--profile release`.


In [ ]:
%%bash
cat > demo_preset.yaml <<'YAML'
dataset:
  provider: wikitext2
  split: validation
  seq_len: 256
  stride: 256
  preview_n: 24
  final_n: 24
  seed: 42
eval:
  metric:
    kind: ppl_causal
YAML

cat > quant_rtn_demo.yaml <<'YAML'
edit:
  name: quant_rtn
  plan:
    bitwidth: 8
    per_channel: true
    group_size: 128
    clamp_ratio: 0.005
    scope: attn
YAML

ls -la demo_preset.yaml quant_rtn_demo.yaml


## Run Compare & Certify


In [ ]:
%%bash
export INVARLOCK_ALLOW_NETWORK=1 INVARLOCK_DEDUP_TEXTS=1 INVARLOCK_TINY_RELAX=1 \
  TRANSFORMERS_NO_TORCHVISION=1 TOKENIZERS_PARALLELISM=false
invarlock certify \
  --baseline gpt2 \
  --subject  gpt2 \
  --adapter  auto \
  --profile  dev \
  --tier     balanced \
  --preset   demo_preset.yaml \
  --out      runs/compare_certify \
  --cert-out reports/compare_certify \
  --edit-config quant_rtn_demo.yaml


## Verify + export HTML


In [ ]:
%%bash
invarlock verify --json reports/compare_certify/evaluation.cert.json
invarlock report html -i reports/compare_certify/evaluation.cert.json -o reports/compare_certify/evaluation.html
echo "Open: reports/compare_certify/evaluation.html"


## Quick interpretation (Python)


In [ ]:
import json

with open('reports/compare_certify/evaluation.cert.json', 'r', encoding='utf-8') as f:
    cert = json.load(f)

print('schema_version:', cert.get('schema_version'))
print('overall_pass:', cert.get('validation', {}).get('overall_pass'))

pm = cert.get('primary_metric', {})
print('primary_metric.kind:', pm.get('kind'))
print('primary_metric.ratio_vs_baseline:', pm.get('ratio_vs_baseline'))

prov = cert.get('provenance', {})
print('provider_digest:', prov.get('provider_digest'))
print('policy_digest:', prov.get('policy_digest'))


## Related docs

- `docs/user-guide/compare-and-certify.md`
- `docs/user-guide/reading-certificate.md`
